In [199]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2018, day=18)

def parses(data):
    # 0=open, 1=trees, 2=lumber
    data = data.replace('.', '0').replace('|', '1').replace('#','2')
    return [[int(i) for i in line] for line in data.strip().split('\n')]
    
data = parses(puzzle.input_data)

In [169]:
sample = parses(""".#.#...|#.
.....#|##|
.|..|...#.
..|#.....#
#.#|||#|#|
...#.||...
.|....|...
||...#|.#|
|.||||..|.
...#.|..|.""")

In [170]:
sample

[[0, 2, 0, 2, 0, 0, 0, 1, 2, 0],
 [0, 0, 0, 0, 0, 2, 1, 2, 2, 1],
 [0, 1, 0, 0, 1, 0, 0, 0, 2, 0],
 [0, 0, 1, 2, 0, 0, 0, 0, 0, 2],
 [2, 0, 2, 1, 1, 1, 2, 1, 2, 1],
 [0, 0, 0, 2, 0, 1, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
 [1, 1, 0, 0, 0, 2, 1, 0, 2, 1],
 [1, 0, 1, 1, 1, 1, 0, 0, 1, 0],
 [0, 0, 0, 2, 0, 1, 0, 0, 1, 0]]

In [184]:
def render(area):   
    N, M = area.shape
    for i in range(N):
        line = ""
        units = []
        for j in range(M):
            c = ['.', '|', '#'][area[i,j]]
            line += c
        line = line + '    ' + ', '.join(units)
        print(line)

In [230]:
def slow_run_step(area):
    new_area = area.copy()
    N, M = area.shape
    for i in range(N):
        for j in range(M):
            counts = [0,0,0]
            for di, dj in itertools.product([-1,0,1], repeat=2):
                if di == 0 and dj == 0:
                    continue
                if 0 <= i+di < N and 0 <= j+dj < M:
                    counts[area[i+di,j+dj]] += 1
            if area[i,j] == 0 and counts[1] >= 3:
                new_area[i,j] = 1
            elif area[i,j] == 1 and counts[2] >= 3:
                new_area[i,j] = 2
            elif area[i,j] == 2 and (counts[1] < 1 or counts[2] < 1):
                new_area[i,j] = 0
    return new_area

In [231]:
def run_step(area):
    new_area = area.copy()
    neigh_slices = list(itertools.product([slice(None,-2), slice(1,-1), slice(2,None)], repeat=2))
    neigh_slices.pop(4) # Remove the center
    p_area = np.pad(area, 1, constant_values=0)
    neigh_trees = np.sum([p_area[s] == 1 for s in neigh_slices], axis=0)
    neigh_lumber = np.sum([p_area[s] == 2 for s in neigh_slices], axis=0)
    new_area[(area == 0) & (neigh_trees >= 3)] = 1
    new_area[(area == 1) & (neigh_lumber >= 3)] = 2
    new_area[(area == 2) & ((neigh_trees < 1) | (neigh_lumber < 1))] = 0
    return new_area

In [228]:
%%timeit
area = np.array(data)
run_step(area)

13.9 ms ± 573 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [229]:
%%timeit
area = np.array(data)
np_run_step(area)

524 µs ± 23.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [235]:
def solve_a(area, mins):
    area = np.array(area)
    for k in range(mins):
        area = run_step(area)
    return (area == 1).sum() * (area == 2).sum()

In [236]:
solve_a(sample, 10)

1147

In [233]:
def solve_b(area):
    seen = {}
    area = np.array(area)
    for k in itertools.count():
        area = run_step(area)
        
        h = hash("|".join("".join([str(c) for c in line]) for line in area))
        if h in seen:
            cycle_len = k - seen[h][0]
            mod_min = (1_000_000_000-k-1) % cycle_len
            return solve_a(area, mod_min)
        value = (area == 1).sum() * (area == 2).sum()
        seen[h] = (k, value)

In [234]:
solve_b(data)

207998